# 🦜🔗 LangChain核心源代码解读：关于LLM和Agent（下）

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# （二）解读源码，深度定制自己的智能体

## 下半场的闲聊

- 🌹 [LangChain 中的 Agent](https://python.langchain.com/docs/modules/agents/agent_types/)
- 🌹 [Chain 遗产](https://python.langchain.com/docs/modules/chains)

### 🌹 智能体相关类整理

**（1）智能体类型**
- AgentType
    - ZERO_SHOT_REACT_DESCRIPTION（`ReAct`的一般实现）
    - REACT_DOCSTORE（ReAct，支持RAG）
    - SELF_ASK_WITH_SEARCH（使用`search 工具`不断反思获得答案）
    - CONVERSATIONAL_REACT_DESCRIPTION（`ReAct`，支持对话）
    - CHAT_ZERO_SHOT_REACT_DESCRIPTION（同上）
    - CHAT_CONVERSATIONAL_REACT_DESCRIPTION（同上）
    - STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION（`ReAct`，为对话模型优化，支持多输入）
    - OPENAI_FUNCTIONS（支持`OpenAI Function Calling`）
    - OPENAI_MULTI_FUNCTIONS（支持`OpenAI Function Calling`，支持多函数调度）
----
**（2）智能体执行器子组件：**

- `AgentAction`
    - `AgentActionMessageLog`
- `AgentStep`
- `AgentFinish`

----
**（3）单动智能体：**
- `BaseSingleActionAgent`
    - RunnableAgent
    - LLMSingleActionAgent（__deprecated__：`create_***_agent`）
    - XMLAgent（__deprecated__：create_xml_agent）
    - Agent
        - ChatAgent（__deprecated__：`create_react_agent`）
        - ConversationalAgent（__deprecated__：`create_react_agent`）
        - ConversationalChatAgent（__deprecated__：`create_json_chat_agent`）
        - StructuredChatAgent（__deprecated__：`create_structured_chat_agent`）
        - ZeroShotAgent（__deprecated__：`create_react_agent`）
        - ReActDocstoreAgent（__deprecated__）
            - ReActTextWorldAgent（__deprecated__）
        - SelfAskWithSearchAgent（__deprecated__：`create_self_ask_with_search`）
    - OpenAIFunctionsAgent（__deprecated__：`create_openai_functions_agent`）

----
**（4）多动智能体：**
- BaseMultiActionAgent
    - RunnableMultiActionAgent
    - OpenAIMultiFunctionsAgent（__deprecated__：`create_openai_tools_agent`）

----
**（5）Assistant：**
- Runnable
    - RunnableSerializable
        - Chain
            - `AgentExecutor`
                - MRKLChain（__deprecated__）
                - ReActChain（__deprecated__）
                - SelfAskWithSearchChain（__deprecated__）
        - `OpenAIAssistantRunnable`

----
**（6）提示语模板：**
- Runnable
    - RunnableSerializable
        - BasePromptTemplate [Dict, PromptValue]
            - BaseChatPromptTemplate
                - ChatPromptTemplate
                    - `AgentScratchPadChatPromptTemplate`

----
**（7）输出解析：**
- Runnable
    - RunnableSerializable
        - BaseOutputParser
            - `AgentOutputParser`
            - `MultiActionAgentOutputParser`

### 🌹 从源码中看 openai 作为先驱的影响

- [partners/openai/langchain_openai/chat_models/base.py（OpenAI模型）](https://github.com/langchain-ai/langchain/blob/master/libs/partners/openai/langchain_openai/chat_models/base.py)
- [community/langchain_community/adapters/openai.py（OpenAI风格API转换）](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/adapters/openai.py)
- [core/langchain_core/utils/function_calling.py（OpenAI风格回调工具函数）](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/utils/function_calling.py)
- [core/langchain_core/output_parsers/openai_tools.py（OpenAI风格输出解析）](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/openai_tools.py)
- [core/langchain_core/output_parsers/openai_functions.py（OpenAI风格输出解析）](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/openai_functions.py)

## 4、 解读 openai 风格智能体

### ✍️ 定义和运行 OpenAI 智能体

In [61]:
# from langchain_openai import ChatOpenAI
from langchain_zhipu import ChatZhipuAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub

def create_neighber(llm):
    """
    使用openai智能体定义一个应用
    """
    # 定义 prompt
    prompt = hub.pull("hwchase17/openai-tools-agent")
    # 定义 tools
    tools = [ask_neighber]
    # 定义 Agent
    agent = create_openai_tools_agent(llm, tools, prompt)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    return executor

neighber = create_neighber(ChatZhipuAI())

### 🌹 观察 OpenAI 智能体的运行过程

#### （1）简单执行：invoke

In [59]:
# invoke
neighber.invoke({"input":"马冬梅住哪里"})



> Entering new AgentExecutor chain...

Invoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`


楼上322根据我的查询结果，马冬梅住在楼上322房间。

> Finished chain.


{'input': '马冬梅住哪里', 'output': '根据我的查询结果，马冬梅住在楼上322房间。'}

#### （2）看看有哪些事件：astream_events

In [14]:
# astream_events
async for e in neighber.astream_events({"input":"马冬梅住哪里"}, version="v1"):
    print(e['name'], e['tags'], e['event'])

AgentExecutor

> Entering new AgentExecutor chain...
 [] on_chain_start
RunnableSequence [] on_chain_start
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_start
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_stream
RunnableParallel<agent_scratchpad> [] on_chain_start
RunnableLambda ['map:key:agent_scratchpad'] on_chain_start
RunnableLambda ['map:key:agent_scratchpad'] on_chain_stream
RunnableParallel<agent_scratchpad> [] on_chain_stream
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_stream
RunnableLambda ['map:key:agent_scratchpad'] on_chain_end
RunnableParallel<agent_scratchpad> [] on_chain_end
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_end
ChatPromptTemplate ['seq:step:2'] on_prompt_start
ChatPromptTemplate ['seq:step:2'] on_prompt_end
ChatZhipuAI ['seq:step:3'] on_chat_model_start
ChatZhipuAI ['seq:step:3'] on_chat_model_stream
ChatZhipuAI ['seq:step:3'] on_chat_model_stream
ChatZhipuAI ['seq:step:3'] on_chat_model_end
OpenAIToolsAgentOutpu

#### （3）观察与大模型的交互过程

- STEP-1 请求智能体（langchain -> ZhipuAI）: 发送带有Tools的请求
- STEP-2 智能体解析（ZhipuAI -> langchain）: 收到Tools-Calling消息
- STEP-3 调用本地工具（langchain）: 调用工具
- STEP-4 重新请求智能体（langchain） -> ZhipuAI）: 提交调用结果
- STEP-5 智能体最终生成（ZhipuAI -> langchain）: 大模型重新生成结果

In [34]:
from langchain_zhipu import ChatZhipuAI
neighber = create_neighber(ChatZhipuAI())

async for e in neighber.astream_events({"input":"马冬梅住哪里"}, version="v1"):
    if e['event'] in ["on_chat_model_end", "on_tool_end"]:
        if("input" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("INPUT:")
            print(e['data']['input'])
        if("output" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("OUTPUT:")
            print(e['data']['output'])
        # print("\n", e)



> Entering new AgentExecutor chain...

 ---------- ChatZhipuAI -- on_chat_model_end ----------
INPUT:
{'messages': [[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='马冬梅住哪里')]]}

 ---------- ChatZhipuAI -- on_chat_model_end ----------
OUTPUT:
{'generations': [[{'text': '', 'generation_info': {'finish_reason': 'tool_calls'}, 'type': 'ChatGenerationChunk', 'message': AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8516985431572319825', 'function': {'arguments': '{"query":"马冬梅住哪里"}', 'name': 'ask_neighber'}, 'type': 'function'}]})}]], 'llm_output': None, 'run': None}

Invoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`


楼上322
 ---------- ask_neighber -- on_tool_end ----------
INPUT:
{'query': '马冬梅住哪里'}

 ---------- ask_neighber -- on_tool_end ----------
OUTPUT:
楼上322

 ---------- ChatZhipuAI -- on_chat_model_end ----------
INPUT:
{'messages': [[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='马冬

### 🌹 观察 OpenAI 智能体的提示语

#### （1）从 Langsmith 的 hub 下载 hwchase17/openai-tools-agent

[查看 hub.pull("hwchase17/openai-tools-agent")](https://smith.langchain.com/hub/hwchase17/openai-tools-agent?organizationId=b877db99-e6ca-519f-9e6c-a297bde66439)

`hub.pull("hwchase17/openai-tools-agent")`并不是获取一个字符串，而是提示语模板。

#### （2）等价的自定义 Prompt 模板

In [50]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

# openai agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

In [58]:
prompt.invoke({
    "input": "请问马冬梅的家在哪里？",
    "chat_history": [],
    "agent_scratchpad": []
})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='请问马冬梅的家在哪里？')])

### ✍️ 一步一步执行，实现 OpenAI 智能体

#### （1）STEP-1 请求智能体（langchain -> ZhipuAI）: 发送带有Tools的请求

In [97]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

# openai agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

input = "马冬梅住在哪里？"
history = []

llm = ChatZhipuAI(tools=[convert_to_openai_tool(ask_neighber)])

agent = prompt | llm

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    convert_to_openai_function 和 convert_to_openai_tool 的区别是什么？
</div>

**🌞 参考：**
- [查看 convert_to_openai_tool 的实现源码](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/utils/function_calling.py#L323-L341)


In [38]:
convert_to_openai_function(ask_neighber)

{'name': 'ask_neighber',
 'description': 'ask_neighber(query: str) -> str - 我在玩找人的游戏，我知道你找的人住在哪个房间',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [39]:
convert_to_openai_tool(ask_neighber)

{'type': 'function',
 'function': {'name': 'ask_neighber',
  'description': 'ask_neighber(query: str) -> str - 我在玩找人的游戏，我知道你找的人住在哪个房间',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string'}},
   'required': ['query']}}}

#### （2）STEP-2 智能体解析（ZhipuAI -> langchain）: 收到Tools-Calling消息

In [104]:
steps_info = []
resp_llm = agent.invoke({"input": input, "chat_history": history, "agent_scratchpad": steps_info})
resp_llm

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8516987149559268121', 'function': {'arguments': '{"query":"马冬梅住在哪里？"}', 'name': 'ask_neighber'}, 'type': 'function'}]})

#### （3）STEP-3 调用本地工具（langchain）: 调用工具

In [105]:
resp_llm.additional_kwargs["tool_calls"][0]["function"]["arguments"]

'{"query":"马冬梅住在哪里？"}'

In [107]:
import json
tool_args = json.loads(resp_llm.additional_kwargs["tool_calls"][0]["function"]["arguments"])
tool_args

{'query': '马冬梅住在哪里？'}

In [108]:
resp_tool = ask_neighber.invoke(tool_args)
resp_tool

'楼上322'

#### （4）STEP-4 重新请求智能体（langchain） -> ZhipuAI）: 提交调用结果

In [110]:
steps_info.append(input)
steps_info.append(resp_llm)
steps_info.append(resp_tool)
steps_info

['马冬梅住在哪里？',
 '楼上322',
 '马冬梅住在哪里？',
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8516987149559268121', 'function': {'arguments': '{"query":"马冬梅住在哪里？"}', 'name': 'ask_neighber'}, 'type': 'function'}]}),
 '楼上322']

#### （5）STEP-5 智能体最终生成（ZhipuAI -> langchain）: 大模型重新生成结果

In [111]:
resp_llm = agent.invoke({"input": input, "chat_history": history, "agent_scratchpad": steps_info})
resp_llm

AIMessage(content='根据我的查询结果，马冬梅住在楼上322。您还需要我提供其他信息吗？')

### 🌹 阅读 create_openai_tool 源码

[langchain/agents/openai_tools/base.py](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/openai_tools/base.py#L15-L97)

```python
def create_openai_tools_agent(
    llm: BaseLanguageModel, tools: Sequence[BaseTool], prompt: ChatPromptTemplate
) -> Runnable:
    """Create an agent that uses OpenAI tools."""

    missing_vars = {"agent_scratchpad"}.difference(prompt.input_variables)
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    llm_with_tools = llm.bind(tools=[convert_to_openai_tool(tool) for tool in tools])

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            )
        )
        | prompt
        | llm_with_tools
        | OpenAIToolsAgentOutputParser()
    )
    return agent
```

### 🌹 阅读 AgentExcutor 源码

## 5、 react风格智能体

### 🌹 阅读 create_react_agent 源码，解读运行过程

- 比原生 Function-Calling 多了必要的解析过程
- 基于提示语可以构建类似原生支持 Function-Calling 的智能体运行逻辑吗？

[查看 create_react_agent 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/react/agent.py#L16)

```python
def create_react_agent(
    llm: BaseLanguageModel,
    tools: Sequence[BaseTool],
    prompt: BasePromptTemplate,
    output_parser: Optional[AgentOutputParser] = None,
    tools_renderer: ToolsRenderer = render_text_description,
    *,
    stop_sequence: Union[bool, List[str]] = True,
) -> Runnable:
    """Create an agent that uses ReAct prompting."""
    
    missing_vars = {"tools", "tool_names", "agent_scratchpad"}.difference(
        prompt.input_variables
    )
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    prompt = prompt.partial(
        tools=tools_renderer(list(tools)),
        tool_names=", ".join([t.name for t in tools]),
    )
    if stop_sequence:
        stop = ["\nObservation"] if stop_sequence is True else stop_sequence
        llm_with_stop = llm.bind(stop=stop)
    else:
        llm_with_stop = llm
    output_parser = output_parser or ReActSingleInputOutputParser()
    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        )
        | prompt
        | llm_with_stop
        | output_parser
    )
    return agent
```

### 🌹 阅读 react 提示语

In [112]:
from langchain_core.prompts import ChatPromptTemplate

prompt_react = ChatPromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

### ✍️ 定义 react 智能体的核心结构

In [113]:
# from langchain_openai import ChatOpenAI
from langchain_zhipu import ChatZhipuAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub

def create_react_neighber(llm):
    # 定义 prompt
    prompt = prompt_react
    # 定义 tools
    tools = [ask_neighber]
    # 定义 Agent
    agent = create_openai_tools_agent(llm, tools, prompt)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    
    return executor

neighber = create_neighber(ChatZhipuAI())

### ✍️ 一步一步测试 create_react_agent 过程

### ✍️ 自定义一个 ReAct 智能体

- 一个简单的智能体需求：与AI玩一个捉迷藏游戏
- react智能体的prompt如何工作
- 中间步骤的一步步产生过程
- action如何解析

## 6、在LCEL框架下重新《手撕AutoGPT》

### ✍️ 自定义《手撕AutoGPT》智能体的提示语
### ✍️ 自定义一个智能体 OutputParser

- 工具：定义一个简单工具
- 智能体：
    - Tools-Calling 智能体
    - ReAct 智能体
- 执行器：AgentExecutor

难点：
- 官方例子和内置智能体无法支持pydantic参数解析（智谱AI等推理能力较弱的模型可以使用）

### 🌹 AgentExcutor中的记忆体

## 7、Langgraph中的惊喜

**一起来打开眼界**

列举示例中对各种新的智能体论文实践...

# 结束了

## ❤️ 总结

- 我们一起阅读了langchain的源代码结构和部份细节
    - BaseLanguageModel / BaseChatModel / 
    - Runnable
    - LambdaRunnable
    - Chain
    - AgentExcutor
    - langgraph.prebuild
- 我们学习了如何自己动手集成大模型到 langchain 中
- 我们拆解了langchain的基石组件：Runnable
- 我们拆解了langchain的核心逻辑能力：LCEL
- 我们拆解了langchain的最新逻辑能力：langgraph
- 我们动手做了一些代码实践